In [1]:
import os
from os.path import join, exists
import sys
import pandas as pd
import numpy as np
import childespy

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

In [2]:
sys.path.append('../../')
from src.utils import load_splits, load_models
from src.analysis import examples_figure
from src.utils import sample_models_across_time, configuration
config = configuration.Config()


%load_ext autotime
%load_ext line_profiler
%pdb

Automatic pdb calling has been turned ON
time: 3.26 ms (started: 2022-06-15 06:34:03 -07:00)


# Success Example

In [3]:
# Make this config regenerate controlled later

pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"', db_version = "2020.1").iloc[0]['id']

regenerate = False
this_path = join(config.prov_csv_dir, 'pvd_utt_glosses.csv')

if regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, type from utterance where corpus_id = '+str(pvd_idx) ,
        db_version = "2020.1")
    utt_glosses.to_csv(this_path, index=False)
else: 
    utt_glosses = pd.read_csv(this_path)

R[write to console]: Using supported database version: '2020.1'.



time: 2.51 s (started: 2022-06-15 06:34:03 -07:00)


## Load phonology and context

In [4]:
all_tokens_phono = load_splits.load_phono()

time: 7.86 s (started: 2022-06-15 06:34:06 -07:00)


In [5]:
success_idx = 16928243

all_tokens_phono.loc[all_tokens_phono.utterance_id == success_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]


,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
997717,I want to read,,,16928243,997717,310,42336
997718,I want to read,aə,aə,16928243,997718,310,42336
997719,I want to read,wɑn,wɑnt,16928243,997719,310,42336
997720,I want to read,də,tu,16928243,997720,310,42336
997721,I want to read,wid,ɹid,16928243,997721,310,42336
997722,I want to read,,,16928243,997722,310,42336


time: 41.2 ms (started: 2022-06-15 06:34:13 -07:00)


In [6]:
target_transcript_id = 42336 # Corresponds to the success_idx 

time: 695 µs (started: 2022-06-15 06:34:14 -07:00)


In [7]:
# Broader context of sentences
utt_glosses.loc[(utt_glosses.transcript_id == target_transcript_id) &
                (utt_glosses.utterance_order.isin(range(310-10, 310+2)))] # Note to self: changed the range here

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
162628,Jasmine,42336,16928069,300,Lily,MOT,declarative
162639,a ballet,42336,16928081,301,Lily,CHI,declarative
162656,is Jasmine a ballerina,42336,16928098,302,Lily,MOT,question
162670,yeah,42336,16928113,303,Lily,CHI,declarative
162686,oh I didn't know that,42336,16928129,304,Lily,MOT,declarative
162710,I ready now yyy,42336,16928154,305,Lily,CHI,declarative
162730,whoa,42336,16928174,306,Lily,MOT,declarative
162744,yyy,42336,16928189,307,Lily,CHI,declarative
162760,yyy yyy,42336,16928205,308,Lily,CHI,declarative
162779,you want mamma let's see,42336,16928225,309,Lily,MOT,declarative


time: 28.7 ms (started: 2022-06-15 06:34:14 -07:00)


## Estimate Prior and Posterior Probability

In [8]:
import imp
imp.reload(examples_figure)
imp.reload(sample_models_across_time)
imp.reload(load_models)

<module 'src.utils.load_models' from '/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/load_models.py'>

time: 4.98 ms (started: 2022-06-15 06:34:14 -07:00)


In [9]:
# how to select the models to run under the new system

time: 5.58 ms (started: 2022-06-15 06:34:14 -07:00)


In [10]:
model_args = load_models.gen_all_model_args() + load_models.gen_unigram_args()

time: 10.1 ms (started: 2022-06-15 06:34:14 -07:00)


In [11]:
raw_scores_across_models = examples_figure.get_scores_across_models(success_idx, model_args, True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running model eval_adult-written_all_no_tags_BERT_Providence_all_0... at age 0
Computing failure scores
Computing success scores
Getting the Levenshtein distance matrix
Running model eval_adult-written_all_no_tags_BERT_Providence_all_0... at age 0
Computing failure scores
Computing success scores


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

Running model eval_adult-written_all_no_tags_BERT_Providence_all_20... at age 0
Computing failure scores
Computing success scores
Getting the Levenshtein distance matrix
Running model eval_adult-written_all_no_tags_BERT_Providence_all_20... at age 0
Computing failure scores
Computing success scores


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

...

Running model eval_no-split_no-dataset_no_tags_flat_unigram_Providence_all_0... at age 0
Getting the Levenshtein distance matrix
Running model eval_no-split_no-dataset_no_tags_flat_unigram_Providence_all_0... at age 0


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

Running model eval_Providence_all_no_tags_data_unigram_Providence_all_0... at age 0
Getting the Levenshtein distance matrix
Running model eval_Providence_all_no_tags_data_unigram_Providence_all_0... at age 0


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

time: 3min 23s (started: 2022-06-15 06:34:14 -07:00)


In [12]:
scores_across_models = pd.concat(raw_scores_across_models)

time: 97.3 ms (started: 2022-06-15 06:37:37 -07:00)


In [13]:
np.unique(scores_across_models.model)

array(['eval_Providence-Age_old_no_tags_BERT_Providence_all_0',
       'eval_Providence-Age_old_no_tags_BERT_Providence_all_20',
       'eval_Providence-Age_old_with_tags_BERT_Providence_all_0',
       'eval_Providence-Age_old_with_tags_BERT_Providence_all_20',
       'eval_Providence-Age_young_no_tags_BERT_Providence_all_0',
       'eval_Providence-Age_young_no_tags_BERT_Providence_all_20',
       'eval_Providence-Age_young_with_tags_BERT_Providence_all_0',
       'eval_Providence-Age_young_with_tags_BERT_Providence_all_20',
       'eval_Providence_all_no_tags_BERT_Providence_all_0',
       'eval_Providence_all_no_tags_BERT_Providence_all_20',
       'eval_Providence_all_no_tags_data_unigram_Providence_all_0',
       'eval_Providence_all_with_tags_BERT_Providence_all_0',
       'eval_Providence_all_with_tags_BERT_Providence_all_20',
       'eval_Switchboard_all_no_tags_BERT_Providence_all_0',
       'eval_Switchboard_all_no_tags_BERT_Providence_all_20',
       'eval_adult-written_all_

time: 4.79 ms (started: 2022-06-15 06:37:37 -07:00)


In [14]:
scores_across_models = scores_across_models.loc[scores_across_models.likelihood_type == 'wfst']

time: 9.72 ms (started: 2022-06-15 06:37:37 -07:00)


In [15]:
childes_all_title = 'eval_Providence_all_with_tags_BERT_Providence_all_20'
adult_all_title = 'eval_adult-written_all_no_tags_BERT_Providence_all_20'
unigram_title = 'eval_Providence_all_no_tags_data_unigram_Providence_all_0'
flat_title = 'eval_no-split_no-dataset_no_tags_flat_unigram_Providence_all_0'

time: 6.29 ms (started: 2022-06-15 06:37:37 -07:00)


In [16]:
success_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','posterior_probability','token','likelihood_type','prior_rank','posterior_rank']]
success_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,posterior_probability,token,likelihood_type,prior_rank,posterior_rank
997721,eval_Providence_all_with_tags_BERT_Providence_...,read weed eat see hear wait with reach what feed,0.990863889601879 0.0028292021781977 0.0011022...,see go play read do eat write come hear talk,0.2788604 0.11265979 0.08179144 0.048948552 0....,0.048949,0.990864,read,wfst,3,0.0


time: 37.6 ms (started: 2022-06-15 06:37:37 -07:00)


In [17]:
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
print('CDL+Context Prior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CDL+Context Prior


'see (0.28) go (0.11) play (0.08) read (0.05) do (0.03) eat (0.02) write (0.02) come (0.01) hear (0.01) talk (0.01)'

time: 23.7 ms (started: 2022-06-15 06:37:37 -07:00)


In [18]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('CDL+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],3))+')' for i in range(len(words))]) 

CDL+Context Posterior


'read (0.991) weed (0.003) eat (0.001) see (0.001) hear (0.001) wait (0.001) with (0.0) reach (0.0) what (0.0) feed (0.0)'

time: 18.7 ms (started: 2022-06-15 06:37:37 -07:00)


In [19]:
success_example = scores_across_models.loc[(scores_across_models.model == adult_all_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example

print('BERT+Context Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

BERT+Context Prior


'read (0.49) see (0.28) play (0.04) know (0.04) look (0.02) go (0.01) watch (0.01) help (0.01) try (0.01) talk (0.01)'

time: 17.6 ms (started: 2022-06-15 06:37:37 -07:00)


In [20]:
adult_all_title

'eval_adult-written_all_no_tags_BERT_Providence_all_20'

time: 8.56 ms (started: 2022-06-15 06:37:37 -07:00)


In [21]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
print('BERT+Context Posterior')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],4))+')' for i in range(len(words))]) 

BERT+Context Posterior


'read (0.9997) weed (0.0001) lead (0.0) see (0.0) wait (0.0) reads (0.0) hear (0.0) eat (0.0) what (0.0) leave (0.0)'

time: 18.2 ms (started: 2022-06-15 06:37:37 -07:00)


In [22]:
success_example = scores_across_models.loc[(scores_across_models.model == unigram_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token', 'prior_rank', 'posterior_rank']]
#print(success_example)
print('CHILDES-1Gram Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1Gram Prior


'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 26.8 ms (started: 2022-06-15 06:37:37 -07:00)


In [23]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('CHILDES-1gram Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

CHILDES-1gram Posterior


'read (0.4) with (0.22) what (0.09) we (0.07) would (0.06) weed (0.02) one (0.01) need (0.01) wheel (0.01) wood (0.01)'

time: 19.3 ms (started: 2022-06-15 06:37:37 -07:00)


In [24]:
success_example = scores_across_models.loc[(scores_across_models.model == flat_title) &
    (scores_across_models.token == 'read')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
success_example
print('UniformPrior Prior')
words = success_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

UniformPrior Prior


'stalks (0.0) rarely (0.0) harbor (0.0) dust (0.0) josh (0.0) milk (0.0) leon (0.0) pitch (0.0) blake (0.0) girlfriend (0.0)'

time: 20.2 ms (started: 2022-06-15 06:37:37 -07:00)


In [25]:
words = success_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in success_example.iloc[0].highest_posterior_probabilities.split(' ')]
print('UniformPrior Posterior')
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

UniformPrior Posterior


'weed (0.53) wheeled (0.08) read (0.08) reed (0.08) wheat (0.04) weeds (0.04) lead (0.02) wood (0.01) would (0.01) with (0.01)'

time: 19.8 ms (started: 2022-06-15 06:37:37 -07:00)


# Failure Example

In [28]:
yyy_idx = 16813515

raw_scores_across_models = examples_figure.get_scores_across_models(yyy_idx, model_args, False)
scores_across_models = pd.concat(raw_scores_across_models)

/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/paths.py:65: ResourceWarning: unclosed file <_io.TextIOWrapper name='/home/stephan/notebooks/child-directed-listening/config.json' mode='r' encoding='UTF-8'>
  config = configuration.Config()
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Running model eval_adult-written_all_no_tags_BERT_Providence_all_0... at age 0
Computing failure scores
Computing success scores
Getting the Levenshtein distance matrix
Running model eval_adult-written_all_no_tags_BERT_Providence_all_0... at age 0
Computing failure scores
Computing success scores


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

Running model eval_adult-written_all_no_tags_BERT_Providence_all_20... at age 0
Computing failure scores
Computing success scores
Getting the Levenshtein distance matrix
Running model eval_adult-written_all_no_tags_BERT_Providence_all_20... at age 0
Computing failure scores
Computing success scores


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

...

Running model eval_no-split_no-dataset_no_tags_flat_unigram_Providence_all_0... at age 0
Getting the Levenshtein distance matrix
Running model eval_no-split_no-dataset_no_tags_flat_unigram_Providence_all_0... at age 0


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

Running model eval_Providence_all_no_tags_data_unigram_Providence_all_0... at age 0
Getting the Levenshtein distance matrix
Running model eval_Providence_all_no_tags_data_unigram_Providence_all_0... at age 0


/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tail[[1]] = -1 * np.log(1)
/home/stephan/notebooks/child-directed-listening/child-directed-listening-env/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/home/stephan/notebooks/child-directed-listening/src/tier_3/../../src/utils/likelihoods.py:312: SettingWithCopyWarning: 
A value is trying to

time: 3min (started: 2022-06-15 06:44:37 -07:00)


In [29]:
all_tokens_phono.loc[all_tokens_phono.utterance_id == yyy_idx][['gloss','actual_phonology_no_dia',
 'model_phonology_no_dia', 'utterance_id','bert_token_id','utterance_order','transcript_id']]

,gloss,actual_phonology_no_dia,model_phonology_no_dia,utterance_id,bert_token_id,utterance_order,transcript_id
289678,you make your yyy,,,16813515,289678,112,42253
289679,you make your yyy,ju,ju,16813515,289679,112,42253
289680,you make your yyy,meək,meək,16813515,289680,112,42253
289681,you make your yyy,jɜ,jɑɹ,16813515,289681,112,42253
289682,you make your yyy,fɜt,*,16813515,289682,112,42253
289683,you make your yyy,,,16813515,289683,112,42253


time: 15.7 ms (started: 2022-06-15 06:47:37 -07:00)


In [30]:
scores_across_models

,prior_rank,prior_prob,entropy,num_tokens_in_context,bert_token_id,set,posterior_entropy,posterior_rank,prior_entropy,position_in_mask,...,highest_posterior_words,highest_prior_words,highest_posterior_probabilities,highest_prior_probabilities,sample_index,age,likelihood_type,model,token,keep
289682,NaN,NaN,3.326737,6.0,289682.0,failure,3.484624,NaN,2.305919,NaN,...,bet feet move shot cut fight fat fit own art,choice point choices move own rounds day mark ...,0.1519765268815855 0.13886932913066247 0.07425...,0.384373 0.29711848 0.09411357 0.030305533 0.0...,0,0,levdist,eval_adult-written_all_no_tags_BERT_Providence...,NaN,NaN
289682,NaN,NaN,3.326737,6.0,289682.0,failure,1.750855,NaN,2.305919,NaN,...,effort fort fortune first fight favorite point...,choice point choices move own rounds day mark ...,0.43808423315301387 0.3293943428132817 0.03217...,0.384373 0.29711848 0.09411357 0.030305533 0.0...,0,0,wfst,eval_adult-written_all_no_tags_BERT_Providence...,NaN,NaN
289682,NaN,NaN,5.121116,13.0,289682.0,failure,2.770887,NaN,3.549688,NaN,...,bet own cut shot bed call guess wish fit move,own choice point bed call guess wish choices m...,0.34480835043287866 0.21079736667097335 0.0651...,0.24905728 0.24319348 0.04453634 0.03318149 0....,0,0,levdist,eval_adult-written_all_no_tags_BERT_Providence...,NaN,NaN
289682,NaN,NaN,5.121116,13.0,289682.0,failure,2.053065,NaN,3.549688,NaN,...,favorite first effort fort offer threat fate f...,own choice point bed call guess wish choices m...,0.35314685480372704 0.2627706130266945 0.15047...,0.24905728 0.24319348 0.04453634 0.03318149 0....,0,0,wfst,eval_adult-written_all_no_tags_BERT_Providence...,NaN,NaN
289682,NaN,NaN,8.498895,7.0,289682.0,failure,3.491548,NaN,5.890984,NaN,...,feet hat foot food bed fit fish cat own head,house bed tower own head dinner name cake cast...,0.270452999933313 0.17445953771463968 0.066023...,0.08865405 0.04948807 0.039698243 0.025803799 ...,0,0,levdist,eval_Providence_all_with_tags_BERT_Providence_...,NaN,NaN
289682,NaN,NaN,8.498895,7.0,289682.0,failure,2.592551,NaN,5.890984,NaN,...,fort favorite first feet front foot face fruit...,house bed tower own head dinner name cake cast...,0.3367189435833728 0.16270389988218012 0.08659...,0.08865405 0.04948807 0.039698243 0.025803799 ...,0,0,wfst,eval_Providence_all_with_tags_BERT_Providence_...,NaN,NaN
289682,NaN,NaN,8.143382,16.0,289682.0,failure,3.543642,NaN,5.644562,NaN,...,feet hat foot food own bed fit fish cat head,own house bed name dinner head tower cake face...,0.22980095767800093 0.1360626141474036 0.09733...,0.11437879 0.05549841 0.03459568 0.028238483 0...,0,0,levdist,eval_Providence_all_with_tags_BERT_Providence_...,NaN,NaN
289682,NaN,NaN,8.143382,16.0,289682.0,failure,2.522041,NaN,5.644562,NaN,...,favorite fort first front foot face feet fruit...,own house bed name dinner head tower cake face...,0.2862257148790275 0.19101842542372083 0.13530...,0.11437879 0.05549841 0.03459568 0.028238483 0...,0,0,wfst,eval_Providence_all_with_tags_BERT_Providence_...,NaN,NaN
289682,NaN,NaN,8.528763,6.0,289682.0,failure,3.198876,NaN,5.911688,NaN,...,feet hat foot food cat fat bed fit head own,house bed tower head own breakfast hat mess mo...,0.2941753891324561 0.17990705575346702 0.12087...,0.09258499 0.038493454 0.035795767 0.026104635...,0,0,levdist,eval_Providence_all_no_tags_BERT_Providence_all_0,NaN,NaN
289682,NaN,NaN,8.528763,6.0,289682.0,failure,2.803192,NaN,5.911688,NaN,...,favorite first foot feet fort face fruit front...,house bed tower head own breakfast hat mess mo...,0.331549323527781 0.09683309619777306 0.087632...,0.09258499 0.038493454 0.035795767 0.026104635...,0,0,wfst,eval_Providence_all_no_tags_BERT_Providence_all_0,NaN,NaN


time: 101 ms (started: 2022-06-15 06:47:37 -07:00)


In [31]:
yyy_example = scores_across_models.loc[(scores_across_models.model == childes_all_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token','likelihood_type','posterior_rank']]
yyy_example

,model,highest_posterior_words,highest_posterior_probabilities,highest_prior_words,highest_prior_probabilities,prior_probability,token,likelihood_type,posterior_rank
289682,eval_Providence_all_with_tags_BERT_Providence_...,favorite fort first front foot face feet fruit...,0.2862257148790275 0.19101842542372083 0.13530...,own house bed name dinner head tower cake face...,0.11437879 0.05549841 0.03459568 0.028238483 0...,NaN,NaN,wfst,NaN


time: 16.1 ms (started: 2022-06-15 06:47:37 -07:00)


In [32]:
words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))])

'own (0.11) house (0.06) bed (0.03) name (0.03) dinner (0.02) head (0.02) tower (0.02) cake (0.02) face (0.01) hand (0.01)'

time: 13.3 ms (started: 2022-06-15 06:47:37 -07:00)


In [33]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'favorite (0.29) fort (0.19) first (0.14) front (0.07) foot (0.06) face (0.05) feet (0.05) fruit (0.02) fist (0.02) fit (0.01)'

time: 16 ms (started: 2022-06-15 06:47:37 -07:00)


In [34]:
yyy_example = scores_across_models.loc[(scores_across_models.model == adult_all_title ) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'own (0.25) choice (0.24) point (0.04) bed (0.03) call (0.03) guess (0.03) wish (0.02) choices (0.02) move (0.02) mistake (0.02)'

time: 14 ms (started: 2022-06-15 06:47:37 -07:00)


In [35]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'favorite (0.35) first (0.26) effort (0.15) fort (0.07) offer (0.02) threat (0.02) fate (0.02) fist (0.01) fit (0.01) freighter (0.01)'

time: 29.3 ms (started: 2022-06-15 06:47:37 -07:00)


In [36]:
yyy_example = scores_across_models.loc[(scores_across_models.model == unigram_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'i (0.04) a (0.03) the (0.03) yeah (0.03) no (0.03) it (0.03) you (0.02) and (0.02) that (0.02) this (0.01)'

time: 18.7 ms (started: 2022-06-15 06:47:38 -07:00)


In [37]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'for (0.41) fit (0.13) first (0.11) front (0.06) foot (0.05) different (0.03) feet (0.03) favorite (0.02) fruit (0.01) forty (0.01)'

time: 17.5 ms (started: 2022-06-15 06:47:38 -07:00)


In [38]:
yyy_example = scores_across_models.loc[(scores_across_models.model == flat_title) & (scores_across_models.likelihood_type == 'wfst')][['model','highest_posterior_words','highest_posterior_probabilities',
    'highest_prior_words','highest_prior_probabilities', 'prior_probability','token']]
yyy_example

words = yyy_example.iloc[0].highest_prior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_prior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'stalks (0.0) rarely (0.0) harbor (0.0) dust (0.0) josh (0.0) milk (0.0) leon (0.0) pitch (0.0) blake (0.0) girlfriend (0.0)'

time: 18 ms (started: 2022-06-15 06:47:38 -07:00)


In [39]:
words = yyy_example.iloc[0].highest_posterior_words.split(' ')
probs = [float(x) for x in yyy_example.iloc[0].highest_posterior_probabilities.split(' ')]
' '.join([words[i]+' ('+str(np.round(probs[i],2))+')' for i in range(len(words))]) 

'freight (0.2) freighter (0.09) fort (0.09) flirt (0.08) front (0.04) fit (0.04) effort (0.03) fate (0.03) favorite (0.02) foot (0.02)'

time: 22.2 ms (started: 2022-06-15 06:47:38 -07:00)


In [40]:
# Get the surrounding context
utt_glosses.loc[(utt_glosses.transcript_id == 42253) &
                (utt_glosses.utterance_order.isin(range(112-3, 112+3)))]

,gloss,transcript_id,id,utterance_order,target_child_name,speaker_code,type
50658,then we won't be able to put them back into th...,42253,16813459,109,Alex,MOT,declarative
50679,do you want ta put some beans in your eggs and...,42253,16813482,110,Alex,MOT,question
50697,no,42253,16813501,111,Alex,CHI,declarative
50710,you make your yyy,42253,16813515,112,Alex,CHI,declarative
50723,can I make one,42253,16813529,113,Alex,MOT,question
50745,no,42253,16813554,114,Alex,MOT,declarative


time: 48.2 ms (started: 2022-06-15 06:47:38 -07:00)
